# Nombres de los estudiantes

Podéis hacer la práctica de forma individual o en parejas. Si trabajáis en parejas basta con que la entregue uno de los dos. 
El profesor puede pediros que defendáis vuestra entrega en cualquier momento para comprobar que la habéis hecho vosotros y la entendéis. La entrega de prácticas demasiado similares o con soluciones que no sabéis explicar puede tener serias consecuencias en la calificación final de la asignatura. 

# Deep Q-Learning

## Entorno

En este notebook vamos a usar el entorno [Cartpole](https://gymnasium.farama.org/environments/classic_control/cart_pole/).

In [1]:
from collections import deque
import numpy as np
import random
import gymnasium as gym
from typing import Tuple
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
env = gym.make('CartPole-v1', render_mode="rgb_array")

print("Espacio de observación:", env.observation_space)
print("  - Shape:", env.observation_space.shape)
print("  - Ejemplo:", env.observation_space.sample())
print()
print("Espacio de acciones:", env.action_space)
print("  - Número de acciones:", env.action_space.n)
print("  - 0: Empujar izquierda")
print("  - 1: Empujar derecha")

In [3]:
env.reset()
plt.axis('off')
plt.imshow(env.render())
display.display(plt.gcf())  
display.clear_output(wait=True)  

## Red neuronal

Vamos a comprobar la versión de pytorch y si tenemos aceleración HW.

Usaremos la variable global DEVICE para mover todos los modelos y datos al dispositivo más rápido antes de operar con ellos.

In [4]:
import torch

print("PyTorch version:", torch.__version__)

DEVICE = 'cuda' if torch.cuda.is_available() \
        else 'mps' if torch.mps.is_available() \
        else 'cpu'

print("Dispositivo disponible:",  DEVICE)

A continuación creamos la red neuronal que usaremos para representar la política.

In [5]:
class DQNNetwork(nn.Module):
    """
    Red neuronal feedforward para aproximar Q-values.
    
    Arquitectura:
    - Capa de entrada: recibe el estado del entorno
    - 2 capas ocultas Linear con activación ReLU
    - Capa de salida Linear: devuelve Q-value para cada acción posible
    
    Parámetros
    ----------
    state_size : int
        Dimensión del espacio de estados (número de features de observación)
    action_size : int
        Número de acciones posibles
    hidden_size : int, opcional
        Número de neuronas en las capas ocultas (default: 128)
    """
    
    def __init__(self, state_size: int, action_size: int, hidden_size: int = 128):
        super(DQNNetwork, self).__init__()

        # TODO TODO
        # Capa de tipo secuencial que contiene las capas de la red
        # Lineal + ReLU + Lineal + ReLU + Lineal
        self.seq = nn.Sequential(
               nn.Linear(state_size,hidden_size),
               nn.ReLU(),
               nn.Linear(hidden_size,hidden_size),
               nn.ReLU(),
               nn.Linear(hidden_size,action_size)
            
        )
        
    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """
        Forward pass de la red.
        
        Parámetros
        ----------
        state : torch.Tensor
            Estado(s) del entorno. Shape: (batch_size, state_size) o (state_size,)
            
        Retorna
        -------
        torch.Tensor
            Q-values para cada acción. Shape: (batch_size, action_size) o (action_size,)
        """
        # TODO TODO
        # Aplica la red neuronal al estado de entrada y devuelve el resultado
        return self.seq(state)


In [6]:
# NO TOCAR
net = DQNNetwork(10, 5, 20)

assert isinstance(net.seq[0], torch.nn.Linear), "Capa 0 debe ser Linear"
assert isinstance(net.seq[1], torch.nn.ReLU), "Capa 1 debe ser ReLU"
assert isinstance(net.seq[2], torch.nn.Linear), "Capa 2 debe ser Linear"
assert isinstance(net.seq[3], torch.nn.ReLU), "Capa 3 debe ser ReLU"
assert isinstance(net.seq[4], torch.nn.Linear), "Capa 4 debe ser Linear"

assert net.seq[0].in_features == 10, "Dimensiones incorrectas"
assert net.seq[0].out_features == 20, "Dimensiones incorrectas"
assert net.seq[2].in_features == 20, "Dimensiones incorrectas"
assert net.seq[2].out_features == 20, "Dimensiones incorrectas"
assert net.seq[4].in_features == 20, "Dimensiones incorrectas"
assert net.seq[4].out_features == 5, "Dimensiones incorrectas"

state = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=torch.float32)
q_values = net(state)
assert torch.equal(q_values, net.seq[4](net.seq[3](net.seq[2](net.seq[1](net.seq[0](state)))))), "error en el método forward"
del state, net, q_values

In [7]:
# Crear una red de ejemplo
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

example_net = DQNNetwork(state_size, action_size, hidden_size=128)
print(example_net)
print()

# Contar parámetros
total_params = sum(p.numel() for p in example_net.parameters())
print(f"Total de parámetros entrenables: {total_params:,}")
print()

# Probar forward pass
example_state = np.array(env.reset()[0])
example_state = torch.FloatTensor(example_state)
q_values = example_net(example_state)
print(f"Estado de entrada shape: {example_state.shape}")
print(f"Q-values de salida shape: {q_values.shape}")
print(f"Q-values: {q_values.detach().numpy()}")
print(f"Acción greedy: {q_values.argmax().item()}")

## Replay Buffer

Necesitamos una memoria donde almacenar las experiencias con las que aprender.

In [8]:
class ReplayBuffer:
    """
    Buffer circular para almacenar y samplear experiencias.
    
    Parámetros
    ----------
    capacity : int
        Tamaño máximo del buffer. Cuando se llena, las experiencias
        más antiguas se eliminan automáticamente.
    """
    
    def __init__(self, capacity: int):
        # TODO TODO
        # Crea un buffer circular usando una cola (deque)
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state: np.ndarray, action: int, reward: float, 
             next_state: np.ndarray, done: bool) -> None:
        """
        Añade una transición al buffer.
        
        Parámetros
        ----------
        state : np.ndarray
            Estado actual
        action : int
            Acción tomada
        reward : float
            Recompensa recibida
        next_state : np.ndarray
            Estado siguiente
        done : bool
            True si el episodio terminó, False en caso contrario
        """
        # TODO TODO
        # Añade al buffer una tupla (estado, acción, recompensa, siguiente estado, terminado)
        self.buffer.append((state,action,reward,next_state,done))

        
    
    def sample(self, batch_size: int) -> Tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
        """
        Samplea un mini-batch aleatorio de experiencias.
        
        Parámetros
        ----------
        batch_size : int
            Número de experiencias a samplear
            
        Retorna
        -------
        Tuple de arrays numpy:
            - states: (batch_size, state_size)
            - actions: (batch_size,)
            - rewards: (batch_size,)
            - next_states: (batch_size, state_size)
            - dones: (batch_size,)
        """
        # Samplear índices aleatorios
        # TODO TODO
        # Usa random.sample para elegir un batch de tuplas
        batch = random.sample(self.buffer,batch_size)
        
        # Desempaquetar el batch en arrays separados
        states, actions, rewards, next_states, dones = zip(*batch)
        
        return (
            np.array(states),
            np.array(actions),
            np.array(rewards, dtype=np.float32),
            np.array(next_states),
            np.array(dones, dtype=np.float32)
        )
    
    def __len__(self) -> int:
        """Retorna el número de experiencias actualmente en el buffer."""
        return len(self.buffer)


In [9]:
# NO TOCAR
buffer = ReplayBuffer(1)
assert len(buffer) == 0, 'el buffer no está vacío inicialmente'
t1 = [1, 2], 3, 4, [5, 6], False
buffer.push(*t1)
assert len(buffer) == 1, 'el método push no almacena las experiencias'
t2 = buffer.sample(1)
for i in range(len(t1)):
    assert (t2[i] == np.array([t1[i]])).all(), 'el método sample no devuelve los datos correctos'
t3 = [10, 20], 30, 40, [50, 60], True
buffer.push(*t3)
assert len(buffer) == 1, 'el buffer no es circular o no tiene el tamaño correcto'

buffer = ReplayBuffer(2)
buffer.push(*t1)
buffer.push(*t3)
assert len(buffer) == 2, 'el método push no almacena las experiencias'
t2 = buffer.sample(1)
for i in range(len(t1)):
    assert (t2[i] == np.array([t1[i]])).all() or (t2[i] == np.array([t3[i]])).all() , 'el método sample no devuelve los datos correctos'
assert len(buffer.sample(2)[0]) == 2, 'el método sample no devuelve el número correcto de experiencias'
del buffer, t1, t2, t3

In [10]:
# Crear buffer de ejemplo
buffer = ReplayBuffer(capacity=1000)

# Llenar con algunas experiencias
state, _ = env.reset()
for _ in range(200):
    action = env.action_space.sample()
    next_state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    
    buffer.push(state, action, reward, next_state, done)
    
    if done:
        state, _ = env.reset()
    else:
        state = next_state

print(f"Experiencias en buffer: {len(buffer)}")
print()

# Samplear un mini-batch
states, actions, rewards, next_states, dones = buffer.sample(batch_size=8)
print("Mini-batch sampled:")
print(f"  States shape: {states.shape}")
print(f"  Actions shape: {actions.shape}")
print(f"  Rewards: {rewards}")
print(f"  Dones: {dones}")

## Selección de la acción

Vamos a implementar una política epsilon greedy con la policy_net.

In [51]:
def epsilon_greedy_policy(policy_net: DQNNetwork, state: np.ndarray, epsilon: float, action_size: int) -> int:
    """
    Selecciona una acción usando epsilon-greedy policy.
    
    Con probabilidad epsilon elige una acción aleatoria (exploración), y con probabilidad 
    1-epsilon elige la  mejor acción según la red (explotación).
    
    Parámetros
    ----------
    policy_net : DQNNetwork
        Red neuronal que representa la política del agente
    state : np.ndarray
        Estado actual del entorno como array
    epsilon : float
        Probabilidad de elegir una acción aleatoria
    action_size : int
        Número de acciones disponibles en el entorno
        
    Retorna
    -------
    int
        Acción seleccionada
    """
    # Exploración: acción aleatoria
    if np.random.random() < epsilon:
        # TODO TODO
        # Devolver acción aleatoria con np.random.randint
        return np.random.randint(0,action_size)
    
    # Explotación: mejor acción según Q-values
    with torch.no_grad():
        # Transformar el estado a un tensor
        state = torch.FloatTensor(state).unsqueeze(0).to(DEVICE)

        # TODO TODO
        # aplicar la red al estado para obtener los valores q
        q_values = policy_net(state)
        
        # devolver el índice del mayor valor Q (como int)
        return q_values.argmax().item()


In [52]:
# NO TOCAR
actions = [epsilon_greedy_policy(None, None, 1, 10) for _ in range(1000)]
assert len(set(actions)) > 1, 'No se devuelven acciones aleatorias con probabilidad epsilon'
assert all(i in actions for i in range(10)), 'No se devuelven todas las acciones posibles del entorno'
policy_net = DQNNetwork(4, 2, 10).to(DEVICE)
state = np.array([1, 2, 3, 4])
actions = [epsilon_greedy_policy(policy_net, state, 0, 10) for _ in range(1000)]
assert len(set(actions)) == 1, "No se devuelve la mejor acción con probabilidad 1 - epsilon"
del actions, policy_net, state

In [53]:
# Elegir una acción
policy = DQNNetwork(4, 2, 10).to(DEVICE)
state = np.array([0.1, 0.2, 0.3, 0.4])
action = epsilon_greedy_policy(policy, state, 0.5, 10)
print("Acción:", action)

# Ahora varias acciones para comprobar que UNA DE ELLAS APARECE MÁS
acciones = [epsilon_greedy_policy(policy, state, 0.5, 10) for _ in range(20)]
print('Acciones:', acciones)

## Paso de entrenamiento

Ahora llegamos a la parte interesante. Un paso de entrenamiento que consiste en:
- Elegir un mini bath de experiencias
- Calcular los valores Q de los estados del minibatch usando la policy_net
- Calcular los nuevos valores Q usando la ecuación de Bellman y la target_net
- Calcular el error entre ambos valores
- Entrenar la red para disminuir el error

In [54]:
def train_step(policy_net: DQNNetwork, 
               target_net: DQNNetwork, 
               gamma: float, 
               optimizer: torch.optim.Optimizer, 
               memory: ReplayBuffer, 
               batch_size: int) -> float:
    """
    Ejecuta un paso de entrenamiento con un mini-batch del replay buffer.
    
    Implementa la actualización de Q-learning:
    Q(s,a) ← Q(s,a) + α[r + γ max_a' Q_target(s',a') - Q(s,a)]
    
    Parámetros
        ----------
        policy_net : DQNNetwork
            Red neuronal que representa la política del agente (entrenada)
        target_net : DQNNetwork
            Red neuronal objetivo (fija temporalmente)
        gamma : float
            Factor de descuento para recompensas futuras
        optimizer : torch.optim.Optimizer
            Optimizador para actualizar los pesos de la red
        memory : ReplayBuffer
            Buffer de experiencias para samplear mini-batches
        batch_size : int
            Tamaño del mini-batch de entrenamiento
        
    Retorna
    -------
    float
        Pérdida (loss) del paso de entrenamiento, o 0.0 si no hay
        suficientes experiencias en el buffer
    """
    # No entrenar si no hay suficientes experiencias
    if len(memory) < batch_size:
        return 0.0
    
    # Samplear mini-batch del replay buffer
    # TODO TODO
    states, actions, rewards, next_states, dones = memory.sample(batch_size)
    assert len(states) == batch_size
    
    # Convertir a tensores de PyTorch
    states = torch.FloatTensor(states).to(DEVICE)
    actions = torch.LongTensor(actions).to(DEVICE)
    rewards = torch.FloatTensor(rewards).to(DEVICE)
    next_states = torch.FloatTensor(next_states).to(DEVICE)
    dones = torch.FloatTensor(dones).to(DEVICE)
    
    # Q-values actuales: Q(s,a) para las acciones tomadas
    # TODO TODO
    # aplicar la policy_net a los estados del mini batch y obtener sus valores Q
    current_q_values = policy_net(states)
    # seleccionar los valores Q de la acción tomada en cada estado
    current_q_values = current_q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
    
    # Q-values objetivo: r + γ max_a' Q_target(s',a')
    with torch.no_grad():
        next_q_values = target_net(next_states).max(1)[0]
        # Si dones = 1 no hay estado siguiente y target_q_values = rewards
        target_q_values = rewards + (1 - dones) * gamma * next_q_values
    
    # Calcular pérdida y actualizar pesos
    # TODO TODO
    # Usa F.mse_loss para calcular el error entre current_q_values y target_q_values
    loss = F.mse_loss(current_q_values,target_q_values)
    
    # minimizar el error
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

Vamos a probar con datos ficticios. El error debería disminuir tras cada paso.

In [55]:
# Es importante que la red y los datos estén en el mismo dispositivo
policy_net = DQNNetwork(4, 2, 10).to(DEVICE)    
target_net = DQNNetwork(4, 2, 10).to(DEVICE)
optimizer = torch.optim.SGD(policy_net.parameters(), lr=0.1)
gamma = 0.9
memory = ReplayBuffer(100)
memory.push([0.1, 0.2, 0.3, 0.4], 0, 1, [0.5, 0.6, 0.7, 0.8], False)
batch_size = 1

loss1 = train_step(policy_net, target_net, gamma, optimizer, memory, batch_size)
print('Error tras 1 paso:', loss1)

loss2 = train_step(policy_net, target_net, gamma, optimizer, memory, batch_size)
print('Error tras 2 pasos:', loss2)

loss3 = train_step(policy_net, target_net, gamma, optimizer, memory, batch_size)
print('Error tras 3 pasos:', loss3)

assert loss1 > loss2 > loss3

## Copia de pesos

Durante el entrenamiento debemos copiar los pesos de la policy_net en la target_net. Vamos a hacer una función para hacerlo.

In [60]:
def copy_weights(policy_net: DQNNetwork, target_net: DQNNetwork) -> None:
    """Copia los pesos de la policy network a la target network."""
    # TODO TODO
    # Usa el método state_dict() para obtener los pesos de una red
    # Usa el método load_state_dict() para establecer los pesos de una red
    
    target_net.load_state_dict(policy_net.state_dict())

In [61]:
# NO TOCAR
policy_net = DQNNetwork(4, 2, 10).to(DEVICE)
target_net = DQNNetwork(4, 2, 10).to(DEVICE)
sd = policy_net.state_dict()
copy_weights(policy_net, target_net)
psd = policy_net.state_dict()
tsd = target_net.state_dict()
assert psd.keys() == sd.keys() and all(torch.equal(psd[k], sd[k]) for k in psd), 'policy_net ha cambiado y no debería'
assert tsd.keys() == sd.keys() and all(torch.equal(tsd[k], sd[k]) for k in tsd), 'target_net no es igual que policy_net'
del policy_net, target_net, sd, psd, tsd

## Entrenamiento

Finalmente vamos a entrenar nuestro agente.

In [62]:
def train(env: gym.Env, n_episodes: int, lr: float, gamma: float, 
          epsilon: float, epsilon_decay: float, epsilon_min: float, 
          memory_size: int, batch_size: int, 
          target_update_steps: int, 
          print_every: int,
          final_model_path: str):
    """
    Entrena un agente DQN en el entorno especificado.

    Parámetros
    ----------
    env : gym.Env
        Entorno de OpenAI Gymnasium.
    n_episodes : int
        Número de episodios de entrenamiento.
    lr : float
        Tasa de aprendizaje para el optimizador.
    gamma : float
        Factor de descuento para recompensas futuras.
    epsilon : float
        Valor inicial de epsilon para la política epsilon-greedy.
    epsilon_decay : float
        Factor de decaimiento de epsilon por episodio.
    epsilon_min : float
        Valor mínimo de epsilon.
    memory_size : int
        Capacidad máxima del replay buffer.
    batch_size : int
        Tamaño del mini-batch para entrenamiento.
    target_update_steps : int
        Frecuencia (en pasos) para actualizar la red objetivo.
    print_every : int
        Frecuencia (en episodios) para imprimir métricas de entrenamiento.

    Retorna
    -------
    policy_net : DQNNetwork
        Red neuronal entrenada que representa la política aprendida.
    """
    
    obs_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    # Redes
    policy_net = DQNNetwork(obs_size, action_size).to(DEVICE)
    target_net = DQNNetwork(obs_size, action_size).to(DEVICE)
    copy_weights(policy_net, target_net)
    target_net.eval()
    
    # Optimizador
    optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr)

    # Replay buffer
    memory = ReplayBuffer(memory_size)

    # Estadísticas
    episode_rewards = []
    episode_losses = []
    avg_rewards = []
    steps = 0

    print("Iniciando entrenamiento...")
    for episode in range(n_episodes):
        state, _ = env.reset()
        episode_reward = 0
        episode_loss = []
        done = False
        while not done:
            # Seleccionar y ejecutar acción
            action = epsilon_greedy_policy(policy_net, state, epsilon, action_size)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            # Almacenar experiencia
            memory.push(state, action, reward, next_state, done)

            # Entrenamiento
            loss = train_step(policy_net, target_net, gamma, optimizer, memory, batch_size)
            steps += 1

            # Actualización de la target_net
            if steps % target_update_steps == 0:
                copy_weights(policy_net, target_net)

            if loss > 0:
                episode_loss.append(loss)
            episode_reward += reward

            state = next_state
        
        # Disminuir epsilon
        epsilon = max(epsilon_min, epsilon * epsilon_decay)

        # Guardar métricas
        episode_rewards.append(episode_reward)
        avg_loss = np.mean(episode_loss) if episode_loss else 0
        episode_losses.append(avg_loss)
        
        # Promedio móvil
        avg_reward = np.mean(episode_rewards[-100:])
        avg_rewards.append(avg_reward)

        # Imprimir progreso
        if episode % print_every == 0:
            print(f"Episodio {episode:4d} | Recompensa: {episode_reward:6.1f} | "
                f"Avg(100): {avg_reward:6.2f} | Epsilon: {epsilon:.3f} | Loss: {avg_loss:.4f}")

    print("\n¡Entrenamiento completado!")
    
    # Salvar modelo
    torch.save(policy_net, final_model_path)
    print(f"\nModelo salvado en {final_model_path}")

    return episode_rewards, avg_rewards, episode_losses, avg_loss

El entrenamiento es muy poco estable. Lo importante es comprobar que la recompensa media de los últimos 100 episodios va aumentando.

Ejecutar la siguiente celda puede llevar un rato... 

In [63]:
import os

os.makedirs("./models/", exist_ok=True)

# Configuración de entrenamiento
env = gym.make('CartPole-v1', render_mode="rgb_array")
n_episodes = 500
lr = 0.001
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
memory_size = 10_000
batch_size = 64
target_update_steps = 20
print_every = 50
final_model_path = 'models/dqn_cartpole'

episode_rewards, avg_rewards, episode_losses, avg_loss = train(
    env, n_episodes, lr, gamma, epsilon, epsilon_decay, epsilon_min, 
    memory_size, batch_size, target_update_steps, print_every, final_model_path)

## 8. Visualizar Resultados del Entrenamiento

In [65]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Gráfica de recompensas
episodes = range(1, len(episode_rewards) + 1)
ax1.plot(episodes, episode_rewards, alpha=0.4, label='Recompensa por episodio')
ax1.plot(episodes, avg_rewards, linewidth=2, label='Promedio móvil (100 episodios)')
ax1.axhline(y=195, color='r', linestyle='--', linewidth=2, label='Objetivo (195)')
ax1.set_xlabel('Episodio', fontsize=12)
ax1.set_ylabel('Recompensa', fontsize=12)
ax1.set_title('Recompensa durante el entrenamiento DQN', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfica de pérdida
ax2.plot(episodes, episode_losses, alpha=0.7, color='orange')
ax2.set_xlabel('Episodio', fontsize=12)
ax2.set_ylabel('Pérdida (MSE)', fontsize=12)
ax2.set_title('Pérdida durante el entrenamiento', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas finales
print(f"\nEstadísticas finales:")
print(f"  Total de episodios: {len(episode_rewards)}")
print(f"  Recompensa máxima: {max(episode_rewards):.0f}")
print(f"  Recompensa promedio (últimos 100): {np.mean(episode_rewards[-100:]):.2f}")

## Vamos a verlo jugar

In [68]:
def play_episode(env: gym.Env, policy_net: DQNNetwork) -> Tuple[int, int]:
    """
    Simula un episodio usando una política e-greedy.

    Parámetros:
        env: entorno de gym.
        policy_net: red que define la política del agente.

    Devuelve:
        Tupla (recompensa_total, pasos_totales) .
    """
    state, info = env.reset()
    episode_steps = episode_reward = 0
    done = False
    while not done:
        # TODO TODO
        # Selecciona la acción adecuada usando la función epsilon_greedy_policy
        # IMPORTANTE: epsilon tiene que ser 0 para que no haga acciones aleatorias
        action = epsilon_greedy_policy(policy_net,state,epsilon=0, action_size=env.action_space.n)

        next_state, reward, terminated, truncated, info = env.step(action)
        episode_steps += 1
        episode_reward += reward
        done = terminated or truncated
        state = next_state
    return episode_reward, episode_steps

In [69]:
# Cargar el modelo entrenado
policy_net = torch.load(final_model_path, weights_only=False)
policy_net.eval()

# Entorno con wrapper de video
env = gym.make('CartPole-v1', render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, episode_trigger=lambda episode_id: True, video_folder='videos/dqn', name_prefix='cartpole')

# Ejecutar una partida 
episode_reward, episode_steps = play_episode(env, policy_net)
print(f"Recompensa del episodio: {episode_reward}")
print(f"Pasos del episodio: {episode_steps}")

env.close()